# Loading packages

In [63]:
#load pacakges 
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install fasttext

import pandas as pd 
import re



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


# Loading, cleaning and filtering data

In [64]:
# Load in the scored csv file
scored_tweets= pd.read_csv('/Users/idahelenedencker/Desktop/STANDBY_Ida/Rec-nition and attack on twitter data/CSV files/twitter_w_rec_attack_final.csv')

#check value counts
scored_tweets['rec-nition'].value_counts(dropna=False) #is 0.0 and 1.0
scored_tweets['attack'].value_counts(dropna=False) #is 0 and 1

#change rec-nition to be instead 0 and 1
scored_tweets['rec-nition'] = scored_tweets['rec-nition'].astype(int)

#remove any word after a @ in the text column
# e.g '@user1122 i dont agree' will be 'i dont agree'
scored_tweets['text'] = scored_tweets['text'].apply(lambda x: re.sub(r'@\w+', '', x).strip())

#filter to replied_to
replies = scored_tweets[scored_tweets['referenced_tweets_type'] == 'replied_to']


In [65]:
replies

,conversation_id,lang,created_at,id,text,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets_id,referenced_tweets_type,in_reply_to_user_id,__index_level_0__,PNR,surveyXact_externke,non_unique_twitter_author_id,started_survey,rec-nition,attack
4,656716588528209920,da,2015-10-21T06:25:53.000Z,6.567180e+17,ja lige ved og næsten.. #MiaSanMia,3.047200e+09,0.0,0.0,0.0,0.0,6.567166e+17,replied_to,1.911496e+09,35.0,7.047328e+08,MV1HVXXWF17V,0.0,0.0,0,0
8,1226455913034403841,da,2020-02-09T21:53:25.000Z,1.226625e+18,Socialist? Svensk socialist?,2.848720e+09,0.0,0.0,0.0,0.0,1.226456e+18,replied_to,2.903894e+09,182.0,2.607603e+09,R1UHY1ULF1EV,0.0,0.0,0,0
10,1225323291948310528,da,2020-02-06T20:34:19.000Z,1.225518e+18,Kommer mer...\nhttps://t.co/psaBVupxlV,2.848720e+09,0.0,0.0,0.0,0.0,1.225323e+18,replied_to,1.202319e+18,273.0,2.607603e+09,R1UHY1ULF1EV,0.0,0.0,0,0
11,1223644293560250368,da,2020-02-01T16:31:28.000Z,1.223645e+18,Smaklig spis på er!,2.848720e+09,0.0,0.0,1.0,0.0,1.223644e+18,replied_to,7.204581e+07,480.0,2.607603e+09,R1UHY1ULF1EV,0.0,0.0,0,0
13,1222099387523964929,da,2020-01-28T10:28:43.000Z,1.222104e+18,I sammanhanget:\nhttps://t.co/ek4HumhWnf,2.848720e+09,0.0,1.0,0.0,0.0,1.222104e+18,replied_to,7.177075e+17,182.0,2.607603e+09,R1UHY1ULF1EV,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1315006,1360151029703208961,da,2021-02-12T09:39:08.000Z,1.360162e+18,Episk klip 💪🏼,3.212709e+08,0.0,0.0,1.0,0.0,1.360151e+18,replied_to,8.613520e+08,3.0,8.107115e+08,S1CH3LRTFNGV,0.0,0.0,0,0
1315007,1348373084605210630,da,2021-01-10T21:29:39.000Z,1.348382e+18,Måske er faktisk bare Shrek...,3.212709e+08,0.0,1.0,0.0,0.0,1.348373e+18,replied_to,1.842927e+07,4.0,8.107115e+08,S1CH3LRTFNGV,0.0,0.0,0,0
1315008,1347102792561909761,da,2021-01-07T09:06:58.000Z,1.347107e+18,* helt tæt på :-),3.212709e+08,0.0,0.0,0.0,0.0,1.347103e+18,replied_to,3.212709e+08,5.0,8.107115e+08,S1CH3LRTFNGV,0.0,0.0,0,0
1315010,1279473024861863941,da,2020-07-05T07:11:34.000Z,1.279674e+18,Fedt og sjovt 👍🏼 (Jeg var 205 km fra),3.212709e+08,0.0,1.0,1.0,0.0,1.279473e+18,replied_to,1.842927e+07,9.0,8.107115e+08,S1CH3LRTFNGV,0.0,0.0,1,0


- conversation_id: refers to the id of the original post
- lang: language
- created_at: when it was created (YYYY-MM-DDTHH:MM:SS.000Z, ISO8601 format giving: year, month, data at Time: hour:minute:second:milisecond in Coordinated Universal Time (UTC))
- id: same as conversation_id unless it is a replied_to
- text: the post, starts with @ if it is a retweet or reply to
- public_metrics.retweet_count: how many retweeted
- public_metrics.reply_count: how many replied
- public_metrics.like_count: how many liked
- public_metrics.quote_count: how many quoted
- referenced_tweets_id: what id this row is a reply or retweet of
- referenced_tweets_type: if it is a post (None), retweeted or replied_to
- in_reply_to_user_id: if it is a replied_to what is the user id of the reciever
- __index_level_0__: ??
- PNR: personnummer (Eentydig identifikation af person), like cpr
- surveyXact_externke: personification ID linked to the e-boks survey
- non_unique_twitter_author_id (made by simon): ?? can take 0 or 1
- started_survey (made by simon): if they started the e-boks survey (doesnt mean finished it) can take 0 or 1

In [66]:
# look at performance of algorithms

#pull up text examples
attack= replies[replies['attack'] == 1]
print(attack['text'][:100])

rec_nition= replies[replies['rec-nition'] == 1]
print(rec_nition['text'][:100])

# can print the whole text using
text_to_print = rec_nition['text'].head(20).tolist()
text_to_print

132      Står i artikeln:"De vanligaste vapnen bland kr...
183                  Svågerpolitiken inom socialdemokratin
264                                         Den svider....
915      Jeg griner stadig... Det er sgu da komisk .. H...
1334     Ja, fanget her. Skulle videre i tog. Må aflyse...
                               ...                        
28377    . har beriget  m madbilleder &amp; skattepolit...
28734    - udlændinge skal ikke have ly i DK, hvis de m...
28989    - nja. Den socialdemokratiske stat er et ekspe...
29069    - når de droppede begge afgifter igen var det ...
29530    - lavere skatter &gt; bedre konkurrenceevne &g...
Name: text, Length: 100, dtype: object
24                                       God Jul på dig 🎅
38                               Ser inget problem i det?
46                             Socialde(m)okrat menar du?
47                                                Billigt
48                                                Billig?
                      

['God Jul på dig 🎅',
 'Ser inget problem i det?',
 'Socialde(m)okrat menar du?',
 'Billigt',
 'Billig?',
 'Vedervärdigt!',
 'Jo, men begriper polisen det?',
 'glæder mig til at læse den. Ses vi fredag på fb ?',
 'nope men det gør jeg. Hvad med Allan ? Vil han gerne med ? Har fundet tøjet til den første temafest. Mangler til den anden..',
 'Tak. Havde jeg lige glemt men skal nok ringe til ham.  Får iøvrigt mit tlf nr tilbage imorgen. kys',
 'Skal jeg nok :0)  Spændende planer denne weekend? Jeg har det lidt svært med ikke at være taget til aarhus',
 'ja en helt igennem fantastisk dejlig dag og aften. Tager gerne en frisk forsyning musik med næste gang vi ses. Bare sig´ til',
 'Enig! Er jo så også en af dem som larmer mens hun lever.... Super klumme. Skru op for musikken!',
 'Åhh sødeste. Al afsked er svær og Berlin er en dejlig dejlig by som du kan vende tilbage til. Nyd denne uge og glæd dig over de..',
 'Selv tak. Har sovet fint men blev i min seng. Den er nu også rar, specielt her om

# Limiting the df to 10.000

## Random sampling

In [67]:
#random sampling
replies_random= replies.sample(10000)
replies_random

,conversation_id,lang,created_at,id,text,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,referenced_tweets_id,referenced_tweets_type,in_reply_to_user_id,__index_level_0__,PNR,surveyXact_externke,non_unique_twitter_author_id,started_survey,rec-nition,attack
1285838,1294927411029708800,da,2020-08-16T13:06:23.000Z,1.294984e+18,SÅ enig. Mange virksomheder har i krisen vist ...,1.914064e+09,0.0,0.0,3.0,0.0,1.294927e+18,replied_to,2.505285e+09,23.0,1.905420e+09,PZKMCT47XZLG,0.0,0.0,1,0
1076302,1410513270486749189,da,2021-07-01T08:41:15.000Z,1.410519e+18,"Nej, Bill finder mig jo aldrig på denne her må...",1.666088e+09,0.0,1.0,2.0,0.0,1.410517e+18,replied_to,1.286995e+18,399.0,1.311571e+09,RQD6MN5D6NGZ,0.0,0.0,0,0
1067634,1366100493383196681,da,2021-02-28T19:00:30.000Z,1.366101e+18,Du er med på det er ligaens bedste defensiv vi...,1.528631e+09,0.0,0.0,0.0,0.0,1.366100e+18,replied_to,1.415454e+08,43.0,8.088724e+08,16AXQGQDZ8CH,0.0,0.0,1,0
1007698,1495432686156095494,da,2022-02-20T16:21:34.000Z,1.495433e+18,Ja det kan vel nærmest være eneste mulige fork...,2.942098e+07,0.0,0.0,0.0,0.0,1.495433e+18,replied_to,2.516316e+09,93.0,1.503741e+09,22XLEYKGXQ3U,0.0,1.0,0,0
138668,94065901158150145,da,2011-07-21T22:55:09.000Z,9.417863e+16,1000 tak - det var overdrevet at være tilbage,1.449997e+08,0.0,0.0,0.0,0.0,9.406590e+16,replied_to,2.794730e+07,133.0,7.029506e+08,AVZ259QZX43D,0.0,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1022146,1373629964931698689,da,2021-03-21T13:44:27.000Z,1.373632e+18,God kamp senere. Vil ud og gå en lang tur i de...,2.942098e+07,0.0,2.0,3.0,0.0,1.373631e+18,replied_to,2.644991e+09,120.0,1.503741e+09,22XLEYKGXQ3U,0.0,1.0,1,0
913132,1017742456421482497,da,2018-07-13T13:26:33.000Z,1.017762e+18,😂😂😂 De havde måtte hente hjælp fra både Sverig...,3.118236e+09,0.0,0.0,0.0,0.0,1.017742e+18,replied_to,3.892381e+07,421.0,NaN,NaN,NaN,NaN,0,0
1103093,1383844358940618758,da,2021-04-18T18:16:29.000Z,1.383847e+18,"Jeg synes også det er indenfor skiven, men jeg...",8.058744e+17,0.0,1.0,1.0,0.0,1.383844e+18,replied_to,3.524030e+09,390.0,NaN,NaN,NaN,NaN,0,0
762013,335087237270220800,da,2013-05-16T18:32:45.000Z,3.351005e+17,NU STOPPER I!,5.467389e+08,0.0,2.0,1.0,0.0,3.351002e+17,replied_to,4.999355e+08,1.0,1.007802e+09,4X3K3ZUHF55G,0.0,0.0,0,0


## Stratified sampling

### Using attack/regnition split

In [68]:
# stratified sampling using attack and rec-nition 

#idea: ¼ are scored 0-0 for attack-recnition; ¼ are scored 1-0 for attack-recnition, ¼ are scored 0-1 for attack-recnition, ¼ are scored 1-1 (possible that there are no/few examples that have both attack AND rec-nition labels) for attack-recnition 

# First: Check if there are any 1-1 cases
condition = (replies['attack'] == 1) & (replies['rec-nition'] == 1)
rows_with_condition = replies[condition]
rows_with_condition #1156 cases (but not enough to be 25%)


#new idea: 1000 of 1-1, 3000 1-0, 3000 0-1, 3000 0-0

#make a new column 'A/R' with both scorings can take 0-0, 0-1, 1-1, or 1-0
replies['A/R'] = replies['attack'].astype(str) + '-' + replies['rec-nition'].astype(str)

#check how many of each
replies['A/R'].value_counts(dropna=False) #is 0 and 1


#Make a new dataframe with the new split 
con_1 = (replies['A/R'] == '1-1')
df1_1 = replies[con_1].sample(1000)

con_2 = (replies['A/R'] == '1-0')
df1_0 = replies[con_2].sample(3000)

con_3 = (replies['A/R'] == '0-0')
df0_0 = replies[con_3].sample(3000)

con_4 = (replies['A/R'] == '0-1')
df0_1 = replies[con_4].sample(3000)

# Concatenate the DataFrames vertically
stratified_sample = pd.concat([df1_1, df1_0,df0_0,df0_1], axis=0)
stratified_sample


/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_19289/994587940.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['A/R'] = replies['attack'].astype(str) + '-' + replies['rec-nition'].astype(str)


,conversation_id,lang,created_at,id,text,author_id,public_metrics.retweet_count,public_metrics.reply_count,public_metrics.like_count,public_metrics.quote_count,...,referenced_tweets_type,in_reply_to_user_id,__index_level_0__,PNR,surveyXact_externke,non_unique_twitter_author_id,started_survey,rec-nition,attack,A/R
1186692,1346890546338537474,da,2021-01-06T18:50:03.000Z,1.346892e+18,Men du har jo ret. Jeg kan godt se problemet m...,3.067945e+08,0.0,2.0,11.0,0.0,...,replied_to,3.527899e+07,365.0,2.306722e+09,GCSNR5WG665Z,0.0,0.0,1,1,1-1
1187262,1328334305899245570,da,2020-11-20T23:14:12.000Z,1.329926e+18,Tak for at bevise du er et egocentrisk r*vhul. 👍😂,3.067945e+08,0.0,1.0,0.0,0.0,...,replied_to,8.742570e+07,95.0,2.306722e+09,GCSNR5WG665Z,0.0,0.0,1,1,1-1
1219759,1285530593699287046,da,2020-07-21T16:38:40.000Z,1.285615e+18,Positivt. Jeg tager også mig selv i tit at sle...,1.565805e+07,0.0,0.0,3.0,0.0,...,replied_to,2.569215e+07,209.0,2.708882e+09,85KM3WZJ1D9Z,0.0,1.0,1,1,1-1
530145,893913212206411778,da,2017-08-06T10:26:30.000Z,8.941426e+17,"Enig, påpeger blot hvordan Klopp udvikler sit ...",3.353206e+08,0.0,1.0,0.0,0.0,...,replied_to,1.081431e+08,169.0,1.110942e+09,XEEF4TNRPHUC,0.0,0.0,1,1,1-1
507139,1145387056828493825,da,2019-06-30T19:29:43.000Z,1.145414e+18,Spørgsmål så - hvor ung/gammel skal man være f...,1.362192e+09,0.0,0.0,1.0,0.0,...,replied_to,1.106040e+09,424.0,2.011761e+09,7FEVME24RYJG,0.0,0.0,1,1,1-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372556,1097762592754421760,da,2019-02-19T19:59:39.000Z,1.097949e+18,Det er klogt. Ellers revner det på et tidspunk...,1.863914e+09,0.0,0.0,1.0,0.0,...,replied_to,3.912452e+08,406.0,1.209771e+09,AU5TQYRXHNGV,0.0,0.0,1,0,0-1
986632,1295410289861689344,da,2020-08-18T11:48:48.000Z,1.295689e+18,Helt enig. Det holder ikke. Jeg tjekker lige o...,8.792361e+07,0.0,2.0,0.0,0.0,...,replied_to,5.559678e+08,100.0,9.088012e+08,MMT7562DRMCV,0.0,0.0,1,0,0-1
1182487,1414285071629049862,da,2021-07-11T18:10:23.000Z,1.414286e+18,Molly er en sød lille Ewok til store Chewie ❤️😍,3.067945e+08,0.0,1.0,2.0,0.0,...,replied_to,1.205545e+09,347.0,2.306722e+09,GCSNR5WG665Z,0.0,0.0,1,0,0-1
156093,855543442486775808,da,2017-04-21T22:15:28.000Z,8.555455e+17,"Den er bestemt interessant, tak",2.044170e+07,0.0,0.0,0.0,0.0,...,replied_to,2.809821e+06,498.0,4.085923e+08,V3UL3XRW625D,0.0,0.0,1,0,0-1


### Using word embeddings

In [ ]:
# preprocessing?? stemming, lowercase, stopwords etc.

#### TF-idf

In [87]:
#cosine similarity to 'had'
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

v= TfidfVectorizer()
X = v.fit_transform(replies['text'])

# Transform the word "had"
word = "had"
word_tfidf = v.transform([word])

# Calculate cosine similarity between the word "had" and each sentence
cosine_similarities = cosine_similarity(X, word_tfidf)

# Output the cosine similarities
replies['cosine_similarity_to_had'] = cosine_similarities


# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='cosine_similarity_to_had', ascending=False)

# Select the top 15 entries
top_15_replies = sorted_replies.head(15)

# Print the results
print(top_15_replies[['text', 'cosine_similarity_to_had']])

/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_19289/637584867.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['cosine_similarity_to_had'] = cosine_similarities


                                                      text  \
852435   Ordet du leder efter er had. MERE HAD MERE HAD!!!   
1014704                        Der var måske ikke nok had.   
50425                          Hvorfor så kalde det “had”?   
874901                            hvorfor så fuld af had??   
1030293  Jeg synes "had" er et stærkt ord. Men jeg er i...   
151393                        Åh, nu er vi tilbage til had   
1137714          Ingen had. Det er bare ikke chokolade.. 😊   
1028746                    Fornemmer efterhånden lidt had!   
876237   Fodbold er følelser, der er kærlighed og der e...   
579952   Lige netop det citat er jo ikke had. Det er en...   
584591             Jeg syntes den var okay. Had, ligefrem?   
42061    "Grådig spekulation". Had, had og atter had. D...   
75443    had jeg forståd vad "primær" betyd had jeg sal...   
869588             #dkmedier - et had/kærlighedsforhold :)   
425466   og det er da fordi, vi netop møder hinanden me...   

       

In [88]:
#cosine similarity to 'politik'

v= TfidfVectorizer()
X = v.fit_transform(replies['text'])

# Transform the word "had"
word = "politik"
word_tfidf = v.transform([word])

# Calculate cosine similarity between the word "had" and each sentence
cosine_similarities = cosine_similarity(X, word_tfidf)

# Output the cosine similarities
replies['cosine_similarity_to_politik'] = cosine_similarities


# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='cosine_similarity_to_politik', ascending=False)

# Select the top 15 entries
top_15_replies = sorted_replies.head(15)

# Print the results
print(top_15_replies[['text', 'cosine_similarity_to_politik']])

                                                      text  \
491361         Deres politik er jo også bare kopi-politik.   
142310   Det er bare politik. Der er ikke fair play i p...   
142313                              Politik. Fordi de kan.   
957860                            Jeg forstår ikke politik   
938001                           God politik, tror jeg! :)   
850238                                   Og en ny politik.   
1247120                 Selvfølgelig! Det er jo politik! 😂   
29502                                    - tal ER politik!   
804650              Politik, hvad er det nu for noget 😄😄😆🤔   
150042   Ingen kan lever den politik, de lover - sådan ...   
489770   selvfølgelig kan man gå til valg på sin politi...   
844864                       Og en ny politik! Tillykke :)   
882488   "for vigtig til politik"? Det er sgu netop nu ...   
859715   flygtninge politik er jo i den grad også EU po...   
394094                              Politik er ikke kønt 🤔   

       

/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_19289/1302457313.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['cosine_similarity_to_politik'] = cosine_similarities


In [90]:
#cosine similarity to 'politik had'

v= TfidfVectorizer()
X = v.fit_transform(replies['text'])

# Transform the word "had"
sentence = "politik had"
sentence_tfidf = v.transform([sentence])

# Calculate cosine similarity between the word "had" and each sentence
cosine_similarities = cosine_similarity(X, sentence_tfidf)

# Output the cosine similarities
replies['cosine_similarity_to_politik_had'] = cosine_similarities


# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='cosine_similarity_to_politik_had', ascending=False)

# Select the top 15 entries
top_15_replies = sorted_replies.head(15)

# Print the results
print(top_15_replies[['text', 'cosine_similarity_to_politik_had']])

/var/folders/ds/0d9wxy210kx_fvknqn3hcg_h0000gn/T/ipykernel_19289/1952813934.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  replies['cosine_similarity_to_politik_had'] = cosine_similarities


                                                      text  \
852435   Ordet du leder efter er had. MERE HAD MERE HAD!!!   
1014704                        Der var måske ikke nok had.   
50425                          Hvorfor så kalde det “had”?   
874901                            hvorfor så fuld af had??   
1030293  Jeg synes "had" er et stærkt ord. Men jeg er i...   
151393                        Åh, nu er vi tilbage til had   
1137714          Ingen had. Det er bare ikke chokolade.. 😊   
1028746                    Fornemmer efterhånden lidt had!   
876237   Fodbold er følelser, der er kærlighed og der e...   
579952   Lige netop det citat er jo ikke had. Det er en...   
491361         Deres politik er jo også bare kopi-politik.   
142310   Det er bare politik. Der er ikke fair play i p...   
142313                              Politik. Fordi de kan.   
957860                            Jeg forstår ikke politik   
584591             Jeg syntes den var okay. Had, ligefrem?   

       

#### pre-tranied BERT-model

In [91]:
!pip install transformers


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import torch

# Sample data
#replies = pd.DataFrame({'text': ["This is a sentence.", "Another example sentence.", "Yet another sentence."]})

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Function to get BERT embeddings
def get_bert_embedding(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

# Get embeddings for all sentences
sentence_embeddings = replies['text'].apply(lambda x: get_bert_embedding(x, tokenizer, model))

# Get embedding for the word "politik"
word = "politik"
word_embedding = get_bert_embedding(word, tokenizer, model).reshape(1, -1)

# Calculate cosine similarity between the word "politik" and each sentence
cosine_similarities = [cosine_similarity([embedding], word_embedding)[0][0] for embedding in sentence_embeddings]

# Output the cosine similarities
replies['cosine_similarity_to_politik_bert'] = cosine_similarities

# Sort the DataFrame by cosine similarity in descending order
sorted_replies = replies.sort_values(by='cosine_similarity_to_politik_bert', ascending=False)

# Select the top 15 entries
top_15_replies = sorted_replies.head(15)

# Print the results
print(top_15_replies[['text', 'cosine_similarity_to_politik_bert']])


In [ ]:
# word embeddings

replies_embeddings=